In [2]:
from unstructured.partition.pdf import partition_pdf

/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# pdf https://www.nerc.com/pa/RAPA/PA/Section1600DataRequestsDL/2024_GADS_Wind_DRI.pdf

pdf_path = "2024_GADS_Wind_DRI.pdf"
elements = partition_pdf(pdf_path, 
                strategy="hi_res",
                infer_table_structure=True,
                extract_images_in_pdf=True,
                max_partition=None)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
len(elements)

903

In [6]:
import json
import pandas as pd

elements_dict = [e.to_dict() for e in elements]
df = pd.DataFrame(elements_dict)
df.head()

,type,element_id,text,metadata
0,Image,389c6ad1643f9f82c5c623f784f06f46,GADS Wind...,{'coordinates': {'points': ((48.05555555555556...
1,Title,bdbfccd772321c20f8d1ac72b2f09c15,GADS Wind Turbine Generation Data Reporting In...,"{'coordinates': {'points': ((150.0, 587.988888..."
2,Title,18de5682df61164c41ade86197ea52c8,"Effective: January 1, 2024","{'coordinates': {'points': ((150.0, 1054.60555..."
3,Image,1d4789feb7bd6d90a4655dcf196facd4,RELIABILITY | RESILIENCE | SECURITY,{'coordinates': {'points': ((806.6666666666666...
4,Image,5f3ee18bb03cd212a1527e27eb064181,"— ' — . i, y \","{'coordinates': {'points': ((47.5, 1624.305555..."


In [8]:
df['type'].value_counts()

type
NarrativeText        270
ListItem             202
Title                191
Table                 61
UncategorizedText     60
Header                55
Formula               34
Image                 22
FigureCaption          8
Name: count, dtype: int64

In [9]:
df[df['type'] == 'Table']['metadata'].iloc[0]

{'detection_class_prob': 0.9014275074005127,
 'coordinates': {'points': ((147.2567901611328, 208.85279846191406),
   (147.2567901611328, 1696.571044921875),
   (1564.552490234375, 1696.571044921875),
   (1564.552490234375, 208.85279846191406)),
  'system': 'PixelSpace',
  'layout_width': 1700,
  'layout_height': 2200},
 'last_modified': '2024-08-28T21:02:27',
 'text_as_html': '<table><tbody><tr><td colspan="2">EVISION HiStOLY...... ce ceceseseceseeseeseesseeeeesecsecseeesecsesseseaeeseecaecsecesecsasseeesseaeessecsecsuecseceseeecadsesesseseseaesaeeeaeseesesesseaseeeeaeesgee iv</td></tr><tr><td colspan="2">FOFACE oo. eececeeecesceceseescsecseeaeseeseacsecsecsseacsecseacsecsesaesacsesaesasseeaseassesaesaesaeseeaeseseesaesassecaeaeseeseeaeeeseesaeeessesaeeaseeeaeeaeeeeaeeatee \\</td></tr><tr><td colspan="2">‘troduction ....</td></tr><tr><td>WhO MUSt REPOFt.....cccccscesecsscsscesccsecssecsecseceseesesessesecesesecseceseseceesesecsecessesecseseseceesesseaeeesseseceeseeeseeseseaeseesesesees

In [10]:
# Convert a single extracted element of type 'Table' to a Markdown table format using 'text_as_html'
def convert_single_table_element_to_markdown(element):
    # Check if the element has 'text_as_html' in its metadata
    if 'text_as_html' not in element:
        raise ValueError("Provided element does not contain 'text_as_html'.")

    # Extract the HTML table from the element
    html_table = element['text_as_html']

    # Use BeautifulSoup to parse the HTML and convert it to Markdown
    from bs4 import BeautifulSoup

    # Parse the HTML
    soup = BeautifulSoup(html_table, 'html.parser')
    markdown_table = ""

    # Extract table headers
    headers = [th.get_text() for th in soup.find_all('th')]
    if headers:
        markdown_table += "| " + " | ".join(headers) + " |\n"
        markdown_table += "| " + " | ".join(['---'] * len(headers)) + " |\n"

    # Extract table rows
    for row in soup.find_all('tr'):
        cells = row.find_all(['td', 'th'])
        if cells:
            markdown_table += "| " + " | ".join(cell.get_text() for cell in cells) + " |\n"

    return markdown_table

# Example usage with a single table element
# markdown_output = convert_single_table_element_to_markdown(single_table_element)
# print(markdown_output)





In [11]:
convert_single_table_element_to_markdown(df[df['type'] == 'Table']['metadata'].iloc[0])

'| EVISION HiStOLY...... ce ceceseseceseeseeseesseeeeesecsecseeesecsesseseaeeseecaecsecesecsasseeesseaeessecsecsuecseceseeecadsesesseseseaesaeeeaeseesesesseaseeeeaeesgee iv |\n| FOFACE oo. eececeeecesceceseescsecseeaeseeseacsecsecsseacsecseacsecsesaesacsesaesasseeaseassesaesaesaeseeaeseseesaesassecaeaeseeseeaeeeseesaeeessesaeeaseeeaeeaeeeeaeeatee \\ |\n| ‘troduction .... |\n| WhO MUSt REPOFt.....cccccscesecsscsscesccsecssecsecseceseesesessesecesesecseceseseceesesecsecessesecseseseceesesseaeeesseseceeseeeseeseseaeseeseseseesesesaeeees | vi |\n| hapter 1: Date Record Types and Forimat.........cccecsscescssesseseeeesseeeeecseesceecseeaeseeseeaeeecsesaeaeseeseeaeseseesaeeessesaeeasesaeeeseeeaeeaees 1 |\n| Data RECOrd Type@s.......eessssscessscsscesceeseeseseesesecsecseesesecseaesessesaeacsecaesacsecseaeseeseeecsecsesaesaesaeseeaesesaesaseessesseeasesseeeseesaeeaeee 1 |\n| FOrMat ....c cee ccccceesecsescsecessceessessceecsecscsecsessesecsessessceecsecsssecssseassssessesessessusesseseessases

In [ ]:
# Function to convert a DataFrame of tables to Markdown format
def convert_dataframe_to_markdown(df):
    markdown_output = ""
    for index, row in df.iterrows():
        if row['type'] == 'Table':
            markdown_output += convert_single_table_element_to_markdown(row['metadata']) + "\n\n" + str(row['metadata']['page_number']) +"\n\n"
            
    return markdown_output

# Example usage with the DataFrame containing multiple elements
markdown_tables = convert_dataframe_to_markdown(df)
print(markdown_tables)


In [5]:
import requests

# Replace with your actual Modal Labs FastAPI endpoint URL
url = "https://jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run/process-pdf/"

# Path to the PDF file on your local machine
pdf_file_path = "2024_GADS_Wind_DRI.pdf"

# Open the PDF file in binary mode
def upload_pdf(endpoint_url, file_path):
    """Uploads a PDF file to the specified endpoint URL."""
    files = {'file': open(file_path, 'rb')}
    response = requests.post(endpoint_url, files=files,verify=False)
    if response.status_code == 200:
        print("PDF uploaded successfully!")
        return response.json()
    else:
        print(f"Error uploading PDF: {response.status_code}")
        print(response.text)
        return None



In [6]:
response = upload_pdf(url, pdf_file_path)

/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run'. Adding certific

PDF uploaded successfully!


In [11]:
import json
# save this as json
with open('response.json', 'w') as f:
    json.dump(response, f)

In [1]:
import json
import pandas as pd
response = json.load(open('response.json'))

In [2]:
from pandas import json_normalize

df = json_normalize(response)

In [3]:
df.head()

,type,text,element_id,metadata.coordinates.points,metadata.coordinates.system,metadata.coordinates.layout_width,metadata.coordinates.layout_height,metadata.filetype,metadata.languages,metadata.page_number,metadata.image_path,metadata.detection_class_prob,metadata.text_as_html,metadata.parent_id
0,Image,GADS Wind...,6cefe70712d1f985da24f15644359efb,"[[48.05555555555556, 53.6111111111113], [48.05...",PixelSpace,1700,2200,application/pdf,[eng],1,/root/figures/figure-1-1.jpg,NaN,NaN,NaN
1,Title,GADS Wind Turbine Generation Data Reporting In...,5427b6f5e20491efb96a5503dd1b5488,"[[150.0, 587.9888888888889], [150.0, 968.27222...",PixelSpace,1700,2200,application/pdf,[eng],1,NaN,NaN,NaN,NaN
2,Title,"Effective: January 1, 2024",8b22337286abf8536ed2c3daf0adb94b,"[[150.0, 1054.6055555555556], [150.0, 1121.272...",PixelSpace,1700,2200,application/pdf,[eng],1,NaN,NaN,NaN,NaN
3,Image,RELIABILITY | RESILIENCE | SECURITY,cc5f020eab26832b0efabe91565cd4db,"[[806.6666666666666, 1565.2222222222222], [806...",PixelSpace,1700,2200,application/pdf,[eng],1,/root/figures/figure-1-2.jpg,NaN,NaN,NaN
4,Image,"ee q — s i, y",10b53442f27c9dae8fbb6e904629ed52,"[[47.5, 1624.3055555555554], [47.5, 1926.38888...",PixelSpace,1700,2200,application/pdf,[eng],1,/root/figures/figure-1-3.jpg,NaN,NaN,NaN


In [4]:
df.columns

Index(['type', 'text', 'element_id', 'metadata.coordinates.points',
       'metadata.coordinates.system', 'metadata.coordinates.layout_width',
       'metadata.coordinates.layout_height', 'metadata.filetype',
       'metadata.languages', 'metadata.page_number', 'metadata.image_path',
       'metadata.detection_class_prob', 'metadata.text_as_html',
       'metadata.parent_id'],
      dtype='object')

In [5]:
df['text']

0                                          GADS  Wind...
1      GADS Wind Turbine Generation Data Reporting In...
2                             Effective: January 1, 2024
3                    RELIABILITY | RESILIENCE | SECURITY
4                                          ee q — s i, y
                             ...                        
896    Outage delayed by pandemic loss of manpower Wo...
897    Outage delayed by pandemic loss of manpower  W...
898                                               Time  
899                                                 Time
900    NERC | GADS Wind Data Reporting Instructions |...
Name: text, Length: 901, dtype: object

In [9]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")

df['text_encoded'] = df['text'].apply(lambda x: enc.encode(x,disallowed_special=()))

In [10]:
df['len_of_text_encoded'] = df['text_encoded'].apply(lambda x: len(x))

In [11]:
df['len_of_text_encoded'].describe()

count    901.000000
mean      39.906770
std       53.883147
min        0.000000
25%       11.000000
50%       21.000000
75%       48.000000
max      471.000000
Name: len_of_text_encoded, dtype: float64

In [12]:
df['metadata.page_number'].value_counts()

metadata.page_number
64    28
26    25
25    25
71    24
72    23
      ..
69     4
63     4
28     3
3      3
29     3
Name: count, Length: 77, dtype: int64

In [14]:
df.groupby('metadata.page_number')['len_of_text_encoded'].sum().values

array([ 79, 365,  99, 567, 209, 326, 363, 206, 455, 413, 578, 568, 609,
       645, 629, 611, 507, 675, 739, 689, 236, 495, 617, 553, 497, 528,
       370,  76,  84, 430, 302, 238, 223, 591, 426, 218, 159, 436, 462,
       160, 526, 576, 756, 525, 486, 899, 673, 403, 397, 644, 722, 217,
       543, 778, 872, 781, 447, 372, 238, 310, 377, 296, 331, 667, 751,
       351, 800, 580,  79, 618, 600, 742, 642, 663, 313, 471,  77])

In [15]:
df.groupby('metadata.page_number')['type'].count().values

array([ 7, 20,  3,  9,  7, 17, 19,  6,  9, 10,  9,  9,  7, 10, 14,  9,  6,
       15, 12, 11,  5, 12, 10, 13, 25, 25, 11,  3,  3, 13,  8,  8,  5, 12,
       12,  8,  7, 15, 17,  6, 16, 16, 13, 15,  8,  7,  5,  5, 12, 13, 12,
        7, 14, 15, 12, 14, 11, 11,  5,  8, 11,  9,  4, 28, 16, 12, 13, 12,
        4, 20, 24, 23, 23, 23, 16, 11,  6])

In [17]:
df[df['metadata.page_number']==77]['text'].values

array(['Appendix N: Examples',
       'Outage delayed by pandemic loss of manpower Work delayed Code: Ext>Pandemic (9015) Outage repair resumes Code: Original cause and type',
       'Outage delayed by pandemic loss of manpower  Work delayed  Code: Ext>Pandemic (9015)  ',
       'Time  ', 'Time',
       'NERC | GADS Wind Data Reporting Instructions | January 2024 71'],
      dtype=object)

In [18]:
df['type'].value_counts()

type
NarrativeText        270
ListItem             202
Title                191
Table                 61
UncategorizedText     58
Header                55
Formula               34
Image                 22
FigureCaption          8
Name: count, dtype: int64

In [21]:
df[df['type']=='ListItem']

,type,text,element_id,metadata.coordinates.points,metadata.coordinates.system,metadata.coordinates.layout_width,metadata.coordinates.layout_height,metadata.filetype,metadata.languages,metadata.page_number,metadata.image_path,metadata.detection_class_prob,metadata.text_as_html,metadata.parent_id,text_encoded,len_of_text_encoded
51,ListItem,1. Configuration Data:,5370f86a49f42ff7121c63decab7517a,"[[180.06666666666666, 948.19775390625], [180.0...",PixelSpace,1700,2200,application/pdf,[eng],6,NaN,0.794155,NaN,a5bab31813f16c2a7db10bcdabdbdf5e,"[16, 13, 24316, 4833, 25]",5
52,ListItem,a. Plant,30025e5b1de028ce183fcd441773824f,"[[228.9783477783203, 1002.4854125976562], [228...",PixelSpace,1700,2200,application/pdf,[eng],6,NaN,0.793668,NaN,a5bab31813f16c2a7db10bcdabdbdf5e,"[64, 13, 23279]",3
53,ListItem,b. Subgroup,9d2a2566e6dfe03bee424c7e7d8e1baa,"[[227.9361572265625, 1056.080322265625], [227....",PixelSpace,1700,2200,application/pdf,[eng],6,NaN,0.818837,NaN,a5bab31813f16c2a7db10bcdabdbdf5e,"[65, 13, 5934, 4323]",4
54,ListItem,c. Energy Storage,8cf7a8613f99faa5dcb96160d096ad03,"[[223.06924438476562, 1110.7158203125], [223.0...",PixelSpace,1700,2200,application/pdf,[eng],6,NaN,0.805270,NaN,a5bab31813f16c2a7db10bcdabdbdf5e,"[66, 13, 16394, 22702]",4
55,ListItem,2. Monthly Performance Data,865ebaaf44205b25ad7ef239d2feb567,"[[178.70672607421875, 1164.1678466796875], [17...",PixelSpace,1700,2200,application/pdf,[eng],6,NaN,0.799403,NaN,a5bab31813f16c2a7db10bcdabdbdf5e,"[17, 13, 62722, 27136, 4833]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,ListItem,Net Maximum Capacity: NMC (Reported in Monthly...,42b56c4df88b8c1f19446389485dbee1,"[[180.06666666666666, 1828.9947509765625], [18...",PixelSpace,1700,2200,application/pdf,[eng],74,NaN,0.828872,NaN,NaN,"[9944, 49848, 63139, 25, 478, 15190, 350, 1448...",12
870,ListItem,2.A.1. Unit Performance Index (PI),1cc5a30774ad4cfd2c8ee7ba6f56ff12,"[[130.0, 248.72222222222243], [130.0, 279.3888...",PixelSpace,1700,2200,application/pdf,[eng],75,NaN,0.409683,NaN,fea47f68673698f5c57d986ef499688d,"[17, 1336, 13, 16, 13, 14856, 27136, 14580, 35...",11
880,ListItem,2.A.3. Net Capacity Factor (NCF),cd96901bbe619f0f8508368cf8d8a073,"[[130.0, 1046.4722222222222], [130.0, 1077.138...",PixelSpace,1700,2200,application/pdf,[eng],75,NaN,0.479516,NaN,NaN,"[17, 1336, 13, 18, 13, 9352, 63139, 63743, 350...",12
892,ListItem,1. The initial delayed period is classified as...,8ffef4f17c54a4ff64733bae2dfe1581,"[[180.06666666666666, 1890.3844444444444], [18...",PixelSpace,1700,2200,application/pdf,[eng],76,NaN,0.878653,NaN,NaN,"[16, 13, 623, 4871, 40480, 5265, 382, 37340, 4...",22


In [32]:
df[df['metadata.page_number']==2][['text','type','metadata.text_as_html']].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>text</th>\n      <th>type</th>\n      <th>metadata.text_as_html</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>7</th>\n      <td>Table of Contents</td>\n      <td>Title</td>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>Who Must Report ....................................................................................................................................................... vi Data Record Types ....................................................................................................................................................... 1 Format ......................................................................................................................................................................... 1 When to Report Configuration Data .........................................................................

In [34]:
from bs4 import BeautifulSoup
def html_table_to_markdown(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    markdown = ""
    
    if table:
        headers = [th.get_text(strip=True) for th in table.find_all('th')]
        rows = [[td.get_text(strip=True) for td in tr.find_all('td')] for tr in table.find_all('tr')]
        
        # Add headers
        markdown += "| " + " | ".join(headers) + " |\n"
        markdown += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        
        # Add rows
        for row in rows:
            markdown += "| " + " | ".join(row) + " |\n"
    
    return markdown

def df_to_markdown(df):
    markdown = ""
    
    for index, row in df.iterrows():
        text = row['text']
        text_type = row['type']
        html_content = row['metadata.text_as_html']
        
        if text_type == 'Title':
            markdown += f"# {text}\n\n"
        elif text_type == 'NarrativeText':
            markdown += f"{text}\n\n"
        elif text_type == 'UncategorizedText':
            markdown += f"**{text}**\n\n"
        elif text_type == 'ListItem':
            markdown += f"- {text}\n"
        elif text_type == 'Image':
            markdown += f"![Image]({text})\n\n"
        elif text_type == 'Table' and pd.notna(html_content):
            markdown += html_table_to_markdown(html_content) + "\n\n"
        else:
            markdown += f"{text}\n\n"
    
    return markdown

In [39]:
markdown_formated = df_to_markdown(df[df['metadata.page_number']==77])

In [40]:
# from rich.pretty import pprint
# pprint(markdown_formated)

'Appendix N: Examples\n\n|  |\n|  |\n| Outage delayed by Work Code: | pandemic loss of manpower delayed Ext>Pandemic (9015) | Outage repair resumes Code: Original cause and type |\n\n\n![Image](Outage delayed by pandemic loss of manpower  Work delayed  Code: Ext>Pandemic (9015)  )\n\n![Image](Time  )\n\n# Time\n\n# NERC | GADS Wind Data Reporting Instructions | January 2024 71\n\n'

In [41]:
from rich.console import Console
from rich.panel import Panel
console = Console()
console.print(Panel(markdown_formated))

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Appendix N: Examples                                                                                            │
│                                                                                                                 │
│ |  |                                                                                                            │
│ |  |                                                                                                            │
│ | Outage delayed by Work Code: | pandemic loss of manpower delayed Ext>Pandemic (9015) | Outage repair resumes  │
│ Code: Original cause and type |                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│ ![Image](Outage delayed by pandemic loss of manpower  Work delayed  Code: Ext>Pandemic (9015)  )                │
│                                                                                                                 │
│ ![Image](Time  )                                                                                                │
│                                                                                                                 │
│ # Time                                                                                                          │
│                                                                                                                 │
│ # NERC | GADS Wind Data Reporting Instructions | January 2024 71                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [50]:
markdown_formated = df_to_markdown(df[df['metadata.page_number']==67])
console = Console()
console.print(Panel(markdown_formated))

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Appendix L: Frequently Asked Questions                                                                          │
│                                                                                                                 │
│ When the value in the denominator is similar between the two equations, the results will be similar. When large │
│ numbers aren’t included in the denominator, the result can be surprising. An example of this is: Plant A has    │
│ 100 turbines, and there are 744 hours in the month. Therefore ACTH = 74,400 hrs. The plant will be down the     │
│ entire month for a planned substation outage. Going into the outage, there is one turbine on FO (Gearbox). The  │
│ gearbox is not repaired during the outage.                                                                      │
│                                                                                                                 │
│ What are equivalent hours? At times, turbines or systems are not out of service but their capacity is reduced.  │
│ During this time, the turbine or system is still accumulating contactor hours. The reduced capacity is          │
│ calculated as equivalent hours and added to the hours in the numerator only. For example, a wind turbine is     │
│ derated 25% for five days. The equivalent hours = 5 days x 24 hrs. /day x 25% = 30 hrs.                         │
│                                                                                                                 │
│ What roll-up method should be used for multiple technologies? Roll‐up calculations for metrics that have        │
│ physical characteristics are straightforward. Simply sum the metric and divide by the number of sites. A metric │
│ with physical characteristics would be dollars, kilowatts, etc. Time metrics represent the length of time in    │
│ which an event occurs. An example of this would be comparing one hour of run time for a 2,000 MW nuclear plant  │
│ to a 1 MW wind turbine. Clearly, one hour of EFOR at the nuclear plant has greater consequences then one hour   │
│ of EFOR at the wind turbine. Another way of looking at this is to compare a 2,000 MW nuclear plant with a wind  │
│ plant with 2,000 1MW wind turbines.                                                                             │
│                                                                                                                 │
│ The pooling equations previously listed work reasonably well when rolling up hourly metrics with turbines of    │
│ the same installed capacity, but errors can be introduced when plant net capacity factors (NCF) vary widely.    │
│ Wind is different from conventional generation in several aspects, such as dealing with large numbers of        │
│ smaller units, variable fuel sources, and temperature constraints. The impact to the wind turbine is no         │
│ different than adjusting a conventional plant’s capacity based upon inlet water temperature cooling capacity.   │
│ The site dependable capacity varies, depending upon the current environment, therefore, there can be a large    │
│ difference between the installed capacity and dependable capacity.                                              │
│                                                                                                                 │
│ Some of the methods for rolling up data into a portfolio metric are:                                            │
│                                                                                                                 │
│ - 1. Averaging: Averaging percentages will only work in certain situations, such as when averaging monthly data │
│ for a single site. In this situation, the pooling equations would be a better choice than averaging             │
│ - 2. Pooling: Pooling equations only work when rolling

In [51]:
def json_to_markdown_by_page(json_data):
    markdown_by_page = {}
    for item in json_data:
        page_number = item['metadata']['page_number']
        text = item['text']
        text_type = item['type']
        html_content = item['metadata'].get('text_as_html')  # Use get() to handle missing keys

        if page_number not in markdown_by_page:
            markdown_by_page[page_number] = ""

        if text_type == 'Title':
            markdown_by_page[page_number] += f"# {text}\n\n"
        elif text_type == 'NarrativeText':
            markdown_by_page[page_number] += f"{text}\n\n"
        elif text_type == 'UncategorizedText':
            markdown_by_page[page_number] += f"**{text}**\n\n"
        elif text_type == 'ListItem':
            markdown_by_page[page_number] += f"- {text}\n"
        elif text_type == 'Image':
            markdown_by_page[page_number] += f"![Image]({text})\n\n"
        elif text_type == 'Table' and pd.notna(html_content):
            markdown_by_page[page_number] += html_table_to_markdown(html_content) + "\n\n"
        else:
            markdown_by_page[page_number] += f"{text}\n\n"

    return markdown_by_page

In [52]:
makrkdown_by_page = json_to_markdown_by_page(response)

In [55]:
console = Console()
console.print(Panel(makrkdown_by_page[71]))

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Appendix M: Equations                                                                                           │
│                                                                                                                 │
│ - 1.A.4. Resource Equivalent Maintenance Outage Factor (REMOF) % of period that the plant was in maintenance    │
│ downtime.                                                                                                       │
│ 𝑅𝐸𝑀𝑂𝐹 = (𝑀𝑇𝐻) 𝐴𝐶𝑇𝐻 × 100                                                                                        │
│                                                                                                                 │
│ - 1.A.5. Resource Equivalent Forced Outage Factor (REFOF)                                                       │
│ % of period that the plant was forced off line. Including low and high winds.                                   │
│                                                                                                                 │
│ 𝑅𝐸𝐹𝑂𝐹 = (𝐹𝑇𝐻 + 𝑅𝑈𝑇𝐻) 𝐴𝐶𝑇𝐻 × 100                                                                                 │
│                                                                                                                 │
│ - 1.A.6. Resource Equivalent Unplanned Outage Factor (REUOF) % of period that the plant was unavailable due to  │
│ forced and maintenance downtime. For generation resource planning.                                              │
│ |  |                                                                                                            │
│ |  |                                                                                                            │
│ |  | REUOF | (FTH = | ACTH | RUTH) -—————_—__— | x | 100 |                                                      │
│                                                                                                                 │
│                                                                                                                 │
│ **× 100**                                                                                                       │
│                                                                                                                 │
│ - 1.A.7. Resource Equivalent Scheduled Outage Factor (RESOF)                                                    │
│ - % of period that the plant was unavailable due to maintenance and planned downtime.                           │
│ 𝑅𝐸𝑆𝑂𝐹 = (𝑀𝑇𝐻 + 𝑃𝑇𝐻) 𝐴𝐶𝑇𝐻 × 100                                                                                  │
│                                                                                                                 │
│ - 1.A.8. Resource Generating Factor (RGF)                                                                       │
│ % of period that the plant was online and in a generating state.                                                │
│                                                                                                                 │
│ 𝑅𝐺𝐹 = (𝐶𝑇𝐻) 𝐴𝐶𝑇𝐻 × 100                                                                                          │
│                                                                                                                 │
│ - 1.A.9. Resource Net Capacity Factor (RNCF) % of actual plant generation.                                      │
│ 𝑅𝑁𝐶𝐹 = 𝑁𝐴𝐺 (𝐴𝐶𝑇𝐻 × 𝑇𝑁𝑀𝐶) × 100                                                                                  │
│                                                                                                                 │
│ # 1.A.10. Net Output Factor (NOF)                                                                               │
│                                                       